In [84]:
import re
from datetime import date

import pandas as pd
import glob
import os

In [85]:
def parse_events_file(file_path: str):
    """Парсит файл событий, сохраняет в структурированный csv-файл и возвращает DataFrame."""
    year_match = re.search(r'(\d{4})', file_path)
    if not year_match:
        raise ValueError("Не удалось определить год из имени файла.")
    year = int(year_match.group(1))

    with open(file_path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]

    MONTHS = {
        "января": 1, "февраля": 2, "марта": 3, "апреля": 4,
        "мая": 5, "июня": 6, "июля": 7, "августа": 8,
        "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12
    }

    pattern = re.compile(
        r"^(\d{1,2})(?:-(\d{1,2}))?\s+([а-яА-ЯёЁ]+)"  #  "20-27 января"
        r"(?:-(\d{1,2})\s+([а-яА-ЯёЁ]+))?"  #  "29-1 марта"
        r"\s*-\s*(.+)$"  #  "20 января - событие"
    )

    rows = []

    for line in lines:
        m = pattern.match(line)
        if m:
            d1, d2, mon1, d3, mon2, event = m.groups()

            date_start_full = None
            date_end_full = None

            try:
                month1 = MONTHS.get(mon1.lower())

                if d3 and mon2:  # случай "29 февраля – 1 марта"
                    month2 = MONTHS.get(mon2.lower())
                    date_start_full = date(year, month1, int(d1))
                    date_end_full = date(year, month2, int(d3))

                elif d2:  # "20–27 января"
                    date_start_full = date(year, month1, int(d1))
                    date_end_full = date(year, month1, int(d2))

                else:  # "20 января"
                    date_start_full = date(year, month1, int(d1))

                rows.append({
                    "date_start": date_start_full,
                    "date_end": date_end_full,
                    "event": event.capitalize()
                })

            except Exception as e:
                print(f'Ошибка при обработке строки: {line}. {e}')

        else:
            print(f'Не удалось распознать строку: {line}')

    df = pd.DataFrame(rows)
    df["date_start"] = pd.to_datetime(df["date_start"])
    df["date_end"] = pd.to_datetime(df["date_end"])
    output_path = re.sub(r'\.txt$', '.csv', file_path)
    df.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"✅ Сохранено {len(df)} событий в {output_path}")
    return df

In [87]:
raw_event_files = glob.glob(os.path.join('./data', "*.txt"))

for file_path in raw_event_files:
    parse_events_file(file_path)

✅ Сохранено 305 событий в ./data\2000_events.csv
✅ Сохранено 238 событий в ./data\2001_events.csv
✅ Сохранено 215 событий в ./data\2002_events.csv
✅ Сохранено 192 событий в ./data\2003_events.csv
✅ Сохранено 138 событий в ./data\2004_events.csv
✅ Сохранено 173 событий в ./data\2005_events.csv
